### 1. Download Google doc as CSV file and upload the file to \content
### 2. Update file_name
### 3. Run all cells

In [ ]:
file_name = '10_16 Deployment Data - Sheet1.csv'

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from openpyxl import Workbook
import ipywidgets as widgets
from IPython.display import display, clear_output

class SoilDataAnalyzer:
    def __init__(self, filename):
        self.data = pd.read_csv(filename, header=None, names=('drop_this', 'Date_Time', 'sensor_vals'))
        self.df = self.data.drop(self.data.columns[[0]], axis=1)

        self.df['parsed_values'] = self.df['sensor_vals'].str.extract(r'{(.*?)}')

        parsed_columns = self.df['parsed_values'].str.split(', ').apply(lambda x: dict(item.split('=') for item in x))
        parsed_df = pd.DataFrame(parsed_columns.tolist(), index=self.df.index)

        self.result_df = pd.concat([self.df, parsed_df], axis=1)
        self.result_df = self.result_df.drop(['parsed_values', 'sensor_vals'], axis=1)
        self.result_df = self.result_df[self.result_df['lid_status'] != '1.0']
        self.result_df['id'] = self.result_df['id'].astype(float).astype(int)

        float_columns = ['air_temp', 'lid_status', 'pressure', 'humidity', 'co2_concentration', 'soil_temp_0', 'moisture_0']
        self.result_df[float_columns] = self.result_df[float_columns].astype(float)

        self.grouped_df = self.result_df.groupby('id')
        self.headers = ['Date_Time', 'air_temp', 'lid_status', 'pressure', 'humidity', 'co2_concentration', 'id', 'soil_temp_0', 'moisture_0']


    def merge_consecutive_rows(self, display_stats=True):
        merged_rows = []
        for group_name, group_data in self.grouped_df:
            merged_group_data = []
            current_row = None
            for _, row in group_data.iterrows():
                if current_row is None:
                    current_row = row
                else:
                    datetime_format = "%m/%d/%Y %H:%M:%S"
                    datetime_obj1 = datetime.strptime(current_row['Date_Time'], datetime_format)
                    datetime_obj2 = datetime.strptime(row['Date_Time'], datetime_format)
                    time_diff = datetime_obj2 - datetime_obj1
                    if time_diff <= timedelta(seconds=60):
                        current_row = current_row.combine_first(row)
                    else:
                        merged_group_data.append(current_row)
                        current_row = row
            merged_group_data.append(current_row)
            merged_rows.extend(merged_group_data)

        self.merged_df = pd.DataFrame(merged_rows)

        self.count_df = self.merged_df.groupby('id').count()
        if display_stats:
            print('Number of data packets from each module:')
            print(self.count_df)
            print('\n=======================================')
            print(f'There are {self.merged_df.isnull().sum().sum()} NaN in the data.')
            for header in self.headers:
                print(f'There are {self.merged_df[header].isnull().sum()} NaN in {header}.')

    def write_data_to_excel(self, excel_filename):
        workbook = Workbook()
        del workbook['Sheet']
        grouped_merged_df = self.merged_df.groupby('id')

        for group_name, group_data in grouped_merged_df:
            sheet = workbook.create_sheet(title=f'ID_{group_name}')

            headers = group_data.columns.tolist()
            sheet.append(headers)

            for row_data in group_data.itertuples(index=False, name=None):
                sheet.append(row_data)

        workbook.save(excel_filename)
        print(f'\nData with headers written to {excel_filename}\n')

    def preprocess_data(self):
        for id, df in self.dfs.items():
            df['Date_Time'] = pd.to_datetime(df['Date_Time'])

    def generate_plot(self, selected_id, selected_y_variable):
        if selected_id and selected_y_variable:
            df = self.dfs[selected_id]
            plt.figure(figsize=(10, 6))
            plt.plot(df['Date_Time'], df[selected_y_variable], marker='.')
            plt.xlabel('Date Time')
            plt.ylabel(selected_y_variable.capitalize())
            plt.title(f'{selected_y_variable.capitalize()} vs Date_Time for {selected_id}')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()

    def interactive_plot(self):
        id_dropdown = widgets.Dropdown(options=self.available_ids, description='Select ID:')
        y_variable_dropdown = widgets.Dropdown(options=self.available_y_variables, description='Select Y-variable:')
        plot_output = widgets.Output()

        def update_plot(change):
            selected_id = id_dropdown.value
            selected_y_variable = y_variable_dropdown.value

            if selected_id and selected_y_variable:
                with plot_output:
                    clear_output(wait=True)
                    self.generate_plot(selected_id, selected_y_variable)

        display(id_dropdown)
        display(y_variable_dropdown)
        display(plot_output)

        id_dropdown.observe(update_plot, 'value')
        y_variable_dropdown.observe(update_plot, 'value')

    def process_data_and_generate_plots(self):
        self.preprocess_data()
        self.interactive_plot()


In [ ]:
analyzer = SoilDataAnalyzer(file_name)

# Merge rows from the same ID if two timestamps are within 60 seconds
analyzer.merge_consecutive_rows()

# Save the data to Excel (output_data.xlsx)
analyzer.write_data_to_excel('output_data.xlsx')

# Read data from the Excel file
analyzer.dfs = pd.read_excel('output_data.xlsx', sheet_name=None)
analyzer.available_ids = list(analyzer.dfs.keys())
analyzer.available_y_variables = ['air_temp', 'pressure', 'lid_status', 'co2_concentration', 'humidity', 'soil_temp_0', 'moisture_0']

# Process data and generate interactive plots
analyzer.process_data_and_generate_plots()

Number of data packets from each module:
    Date_Time  humidity  lid_status  air_temp  co2_concentration  pressure  \
id                                                                           
2           1         1           1         1                  1         1   

    moisture_0  soil_temp_0  
id                           
2            1            1  

There are 0 NaN in the data.
There are 0 NaN in Date_Time.
There are 0 NaN in air_temp.
There are 0 NaN in lid_status.
There are 0 NaN in pressure.
There are 0 NaN in humidity.
There are 0 NaN in co2_concentration.
There are 0 NaN in id.
There are 0 NaN in soil_temp_0.
There are 0 NaN in moisture_0.

Data with headers written to output_data.xlsx



Dropdown(description='Select ID:', options=('ID_2',), value='ID_2')

Dropdown(description='Select Y-variable:', options=('air_temp', 'pressure', 'lid_status', 'co2_concentration',…

Output()